In [1]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

mydata = spark.read.csv('/FileStore/tables/CCPP.csv', header=True, inferSchema=True)
mydata.show(1)

In [2]:
vectorassembler = VectorAssembler(inputCols=["AT","V","AP","RH"], outputCol = "features")
vec_df = vectorassembler.transform(mydata)
vec_df.show(1)

In [3]:
traintestsample = vec_df.randomSplit([0.7,0.3],seed= 123)
train_df = traintestsample[0]
test_df = traintestsample[1]
train_df.count()

In [4]:
test_df.count()

In [5]:
dt = DecisionTreeRegressor(featuresCol="features", labelCol="PE")
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)

In [6]:
dt_evaluator= RegressionEvaluator(labelCol="PE",predictionCol="prediction",metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
rmse

In [7]:
##decisiontreeclassification

from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer


mydata = spark.read.csv('/FileStore/tables/iris.txt', inferSchema = True)
mydata.show(1)

In [8]:
mydata = mydata.select(col("_c0").alias("sepal_length"),
                       col("_c1").alias("sepal_width"),
                       col("_c2").alias("petal_length"),
                       col("_c3").alias("petal_width"),
                       col("_c4").alias("specis"))



In [9]:
mydata.take(1)
mydata.show(1)

In [10]:
vectorassembler = VectorAssembler(inputCols=["sepal_length","sepal_width","petal_length","petal_width"],outputCol="features")
vector_iris = vectorassembler.transform(mydata)
vector_iris.show(2)

In [11]:
##stringindexer used for indexing categorical variables
indexer = StringIndexer(inputCol="specis",outputCol="label")
indexed_vectorized_df = indexer.fit(vector_iris).transform(vector_iris)
indexed_vectorized_df.show(1)

In [12]:
##splitting up train and test

traintestsample = indexed_vectorized_df.randomSplit([0.7,0.3],seed=123)
train_df = traintestsample[0]
test_df = traintestsample[1]
train_df.count()
train_df.show(30)

In [13]:
test_df.count()

In [14]:
##model building

from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="label",featuresCol="features")

dt_model = dt.fit(train_df)

dt_predictions = dt_model.transform(test_df)

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
dt_evaluator = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
dt_accuracy = dt_evaluator.evaluate(dt_predictions)
dt_accuracy